In [2]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr"] }
:dep qlab = { path = "D:/dev/qlab" }
:dep chrono
:dep time

In [3]:
use plotters::prelude::*;
use qlab::{act365, DiscountCurve, InterpolationType, ZeroCurve};
use chrono::NaiveDate;
use time::Duration;

In [5]:

let anchor: NaiveDate = NaiveDate::from_ymd(2020, 1, 1);
let mut dfs = vec![1f64; 365*5];
let mut ds: Vec<NaiveDate> = vec![];
{
    for i in 0..dfs.len() {
        let d = anchor + Duration::days(i as i64);
        ds.push(d)
    }

    let dates: Vec<NaiveDate> = vec![
        NaiveDate::from_ymd(2021, 1, 1),
        NaiveDate::from_ymd(2022, 1, 1),
        NaiveDate::from_ymd(2023, 1, 1),
        NaiveDate::from_ymd(2024, 1, 1),
    ];
    let rates = vec![0.01, 0.02, 0.03, 0.04];    
    let zc = ZeroCurve::new(
        anchor,
        dates.clone(),
        rates,
        act365,
        vec![
            InterpolationType::Linear,
            InterpolationType::Linear,
            InterpolationType::Linear,
        ],
        vec![3, 3],
    )?;
    zc.dfs(&ds, &mut dfs)?;
}

let mut fs = vec![0.;dfs.len()-1];
for i in 1..dfs.len() {
    fs[i-1] = (dfs[i-1]/dfs[i]-1.)*365.
}
evcxr_figure((640, 240), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Flat forward curve!", ("Arial", 20).into_font())
        .build_ranged(0..fs.len(), 0.01f64..0.1f64)?;
    // Then we can draw a series on it!
    chart.draw_series(fs.iter().enumerate().map(|(i, &df)|{
        Circle::new((i, df), 1, &RED)
    }))?;
    Ok(())
}).style("")

Flat forward curve!

In [ ]:
//println!("{:?} {:?}", ds, fs)